# CSC 480-F23 LandscAIpe Demo - Deep Learning Object Segmentation with Mask R-CNN



## Version history

- Version 1: Fall 2023

# Authors:


***Nathan Tran***
***Ethan Outangoun***
***Skyler Han***
***Saanvi Dua***
***Christine Widden***

California Polytechnic State University, San Luis Obispo;

Computer Science & Software Engineering Department

## Project Description

This document explores the

```
# Explain what this document is about, and what it does.
# Instead of "Abstract", you could also label it "Project Description", or leave out the label.
# After reading this, the potential user should be able to make a decision about engaging further with it, or go elsewhere.
# Point out the benefits and value for the prospective user.
# If applicable, identify the main technologies, tools, libraries, data sets, other resources that it will use.
# Identify the sources and creators of those resources, usually through links to their respective Web sites, or references to publications.

```


# Copyright: Apache 2.0 License


```
# By posting a notebook like this on the Web, you are making it publicly accessible, and it's hard to control who uses it how.
# To make this more explicit, I suggest to declare it as open source, and specify the license and main conditions under which it can be used.
# This won't prevent unauthorized use, but at least your intentions are clear.  
```



```
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

```


# Colab Basics




## Colab and Jupyter Notebooks


### Setting up Conda Environment

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

## Setting Up Your Drive

First run the command below to link your google drive to this colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, you are going to want to create a new folder in your drive called "Mask_RCNN Demo Final". After creating the folder, you will want to access the new folder you just created. Type in the following and run the cell

In [ ]:
%cd drive/My Drive/Mask_RCNN Demo

Now we will clone the mask_rcnn repository from github into this new project folder. Type in the following and run the cell


In [ ]:
! git clone https://github.com/ahmedfgad/Mask-RCNN-TF2.git

Now, you will want to rename the folder "kangaro-transfer-learning" as as "building-transfer-learning". After renaming that, move this google colab notebook into the folder.

After moving the notebook, you will want to access the Mask-RCNN-TF2 folder in your drive. Run the following cell

In [ ]:
%cd Mask-RCNN-TF2

You will need to set up your environment now and install all of the needed dependencies. Run the following cell that will install of the requirements.

In [31]:
! python -m pip install -r 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


Now your set up is complete and you can move onto actually training your model!

In [ ]:
!conda install gdal
!apt-get install -y libgdal-dev
!conda install geoio

# Overview

This section gives the participants a preview of the activity they are going to work on. It should be short, maybe one paragraph or so. If more information is needed, add extra sections below.

## Prerequisites

Basic knowledge of linux commands and python is necessary. The tutorial is pretty simple and setting up the environment is mainly done through running the cells.

##Learning Objectives

*   Understand a basic concept of mask_rcnn.
*   Be able to edit and train a machine learning model.
*   Mask an image with the trained model using testing images.



# Data Set


The dataset we will be using is from SpaceNet.

## Data Preparation

### Exploration of the Data

The dataset is pretty simple. It is a bunch of sattelite images of homes. However, there is a decent amount of preprocessing we are going to have to do. For mask_rcnn to train, we are going need to have to create json tags for all of the images.

# Preprocessing

Mask R-CNN requires there to be labeled json files for each image and for the image to be ordered alphabetically for training. We will use methods developed by Mstfakts to preprocess all of our image files from SpaceNet-AI.

In [ ]:
#Access building-transfer-learning folder
%cd building-transfer-learning

### Renaming RGB-PanSharpen and GeoJson Files

We will need to run the Rename_Files.ipynb. Without running this file, you will get errors because of the naming of the training files. Moreover, you will also need to order the training files by its name as they are unordered. Run the following cell to do this.


In [39]:
import os
import numpy as np
import re
PATH = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen/'
Geolar = os.listdir(PATH)
for i in range(len(Geolar)):
    oldNameGeo = os.path.join(PATH, Geolar[i])
    match = re.search(r'(\d+)\.tif$', Geolar[i])  # Extract digits before ".tif" at the end
    if match:
        number = match.group(1)
        newNameGeo = os.path.join(PATH, 'tif' + number + '.tif')
        os.rename(oldNameGeo, newNameGeo)

In [50]:
import os
import numpy as np
PATH = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/geojson/buildings/'
Geolar = os.listdir(PATH)
for i in range(len(Geolar)):
    oldNameGeo = os.path.join(PATH, Geolar[i])
    match = re.search(r'(\d+)\.geojson$', Geolar[i])  # Extract digits before ".tif" at the end
    if match:
        number = match.group(1)
        newNameGeo = os.path.join(PATH, 'geojson' + number + '.geojson')
        os.rename(oldNameGeo, newNameGeo)

### Convert RGB-PanSharpen Tif Files to PNG

We have the dataset and its files' name are in order, but its format it TIFF. So, convert the TIFF file to the RGB file by running TIF_to_PNG.py. Run the following cell below to do this

In [42]:
# -*- coding: utf-8 -*-

from osgeo import gdal
import numpy as np
import os
import subprocess

def bit16_to_8Bit(inputRaster, outputRaster, outputPixType='Byte', outputFormat='png', percentiles=[2, 98]):
    '''
    Convert 16bit image to 8bit
    Source: Medium.com, 'Creating Training Datasets for the SpaceNet Road Detection and Routing Challenge' by Adam Van Etten and Jake Shermeyer
    '''

    srcRaster = gdal.Open(inputRaster)
    cmd = ['gdal_translate', '-ot', outputPixType, '-of',
           outputFormat]

    # iterate through bands
    for bandId in range(srcRaster.RasterCount):
        bandId = bandId+1
        band = srcRaster.GetRasterBand(bandId)

        bmin = band.GetMinimum()
        bmax = band.GetMaximum()
        # if not exist minimum and maximum values
        if bmin is None or bmax is None:
            (bmin, bmax) = band.ComputeRasterMinMax(1)
        # else, rescale
        band_arr_tmp = band.ReadAsArray()
        bmin = np.percentile(band_arr_tmp.flatten(),
                             percentiles[0])
        bmax= np.percentile(band_arr_tmp.flatten(),
                            percentiles[1])

        cmd.append('-scale_{}'.format(bandId))
        cmd.append('{}'.format(bmin))
        cmd.append('{}'.format(bmax))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))
    cmd.append(inputRaster)
    cmd.append(outputRaster)
    print("Conversin command:", cmd)
    subprocess.call(cmd)

path = "drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen/"
files = os.listdir(path)

for file in files:
    resimPath = path+file
    dstPath   = "drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen-NEW/"
    dstPath   = dstPath+file
    bit16_to_8Bit(resimPath,dstPath)

Conversin command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-scale_1', '0.0', '1275.0', '0', '255', '-scale_2', '0.0', '1772.0', '0', '255', '-scale_3', '0.0', '1249.0', '0', '255', 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen/image36.tif', 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen-NEW/image36.tif']
Conversin command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-scale_1', '0.0', '677.0', '0', '255', '-scale_2', '0.0', '857.0', '0', '255', '-scale_3', '0.0', '600.0', '0', '255', 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen/image35.tif', 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen-NEW/image35.tif']
Conversin command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-scale_1', '0.0', '640.0', '0', '255', '-scale_2', '0.0', '776.0', '0', '255', '-scale_3', '0.0', '549.0', '0',

### Create Labels for TIFF and GeoJson Files

To train the model, we need the labels, too. We will create these labels by using TIFF files and its corresponding GeoJSON files. We need the TIFF file to adjust the position of the GeoJSON coordinates to the specific picture. You only need to follow Create_Masks.ipynb

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import geoio

In [51]:
ResimPATH = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen/'
# All the pictures from the given path
ResimAdlari = os.listdir(ResimPATH)
GeoJSONPATH = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/geojson/buildings/'
# All the geojson-files from the given path
GeoAdlari = os.listdir(GeoJSONPATH)

In [52]:
# DosyaNumarasi takes the number from 0 to how big your training dataset
for DosyaNumarasi in range(len(GeoAdlari)):

    # data holds the info for the each GeoJSON-file
    with open(GeoJSONPATH+'/'+GeoAdlari[DosyaNumarasi]) as f:
        data = json.load(f)

    # RGBTIFResmi holds the info for each TIF-file
    RGBTIFResmi = geoio.GeoImage(ResimPATH+'/'+ResimAdlari[DosyaNumarasi])

    cokgenler = [] # Hold the coordinates for each building in the picture.
                   # (Outside for loop is for each picture, and here, cokgenler
                   # will hold the coordinates for each building in one picture.)
    types = [] # Holds the type of the buldings (MultiPolygon - Partial Building - Point)
               # We are not interested in the points.

    # Create the pane size of 650x650 to put the figures from geojson-file. Otherwise,
    # the buildings may be flipped or they may saved one by one
    arkaPlan = np.zeros([650,650])
    plt.imshow(arkaPlan)

    try:
        # We do not know how many buildings the picture includes.
        # So, we just give very big number to make sure that we utilized
        # all the buildings in one picture.
        # In short, bina keeps what order the building is.
        for bina in range(2000):
            tip = str(data['features'][bina]['geometry']['type'])
            types.append(tip) # Append all the type of the buildings

            # If type is point, do not do anything
            if tip == ('Point'):
                pass

            # If type is MultiPolygon, cokgenler will hold the coordinates
            elif tip == ('MultiPolygon'):
                kucukBinalar = (data['features'][bina]['geometry']['coordinates'])
                for b in range(len(kucukBinalar)):
                    cokgenler.append(kucukBinalar[b])

            # For the rest of the types, cokgenler will hold the coordinates again
            else:
                cokgenler.append(data['features'][bina]['geometry']['coordinates'])

    except IndexError:
        # If we utilized all the buildings in the given picture,
        # lest create mask for each one.

        # cokgenBina holds the each building's coordinates
        for cokgenBina in cokgenler:

            # binaNoktalari holds the individual edge coordinates for each building.
            for binaNoktalari in cokgenBina:

                # To hold the edge coordinates (in pixel form)
                doldurX = []
                doldurY = []

                # noktas holds x and y for each edge coordinate
                for noktas in binaNoktalari:

                    # Convert Latitude&Longitude to the pixels
                    xPixel, yPixel = RGBTIFResmi.proj_to_raster(noktas[0], noktas[1])

                    # The pixels may be 650 which defaces the masks.
                    xPixel = 649 if xPixel > 649 else xPixel
                    yPixel = 649 if yPixel > 649 else yPixel

                    # Keep x and y in pixel form
                    doldurX.append(xPixel)
                    doldurY.append(yPixel)

                # To paint between given pixel values
                plt.fill_between(doldurX, doldurY, facecolor='red')

                # To remove white area around matplotlib figure
                fig = plt.figure(1)
                extent = plt.gca().get_window_extent().transformed(fig.dpi_scale_trans.inverted())

                # Adjust the DPI for 650x650
                # and save the figure
                # While saving, you should put them in order; 0 to ...
                fig.savefig('drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/MASKS/'+str(DosyaNumarasi)+'.png', bbox_inches=extent, dpi=215.24)

        # Close the figure after an image is done.
        plt.close()

### Display GeoJSON

In [53]:
import geopandas as gpd #To read geojson file
import matplotlib.pyplot as plt #To display the geojson file
%matplotlib inline

In [55]:
geoFileInfo = gpd.read_file('drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/geojson/buildings/image_geojson1.geojson')

In [ ]:
geoFileInfo #Exhibit the values

In [ ]:
geoFileInfo.plot(color='black') # Plot the file

### Display Mask and RGB Images

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [68]:
image_id = 1 # Give the image ID

In [69]:
ResimAdi = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/RGB-PanSharpen-NEW/image'+str(image_id)+'.tif'
imggg = Image.open(ResimAdi).convert("L")
numpyResim = np.array(imggg)
imggg.close()

ResimPATH = 'drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/MASKS/'+str(image_id)+'.png'
imggg = Image.open(ResimPATH)
imgarr = np.array(imggg)
imggg.close()

fig = plt.figure(1, figsize = [20, 10], dpi = 200)
plt.subplot(1,2,1)
plt.imshow(imgarr)
plt.subplot(1,2,2)
plt.imshow(numpyResim)
plt.show()

FileNotFoundError: ignored

# Model

This term is often used in connection with Machine Learning tools and refers to a specific configuration of a network (e.g., ResNet or YOLOv7). For other tools, you can describe the system architecture in terms of the main components it uses, and how they interact.

## Model Selection

## Model Design

Most often, the model will be based on an existing architecture, and built from components of a framework like PyTorch or TensorFlow.

## Model Training

To train the model, run the following cell

In [65]:
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

#Libraries' order is important
import geopandas as gpd
# Geoio dan önce geopandası yüklemelisin.
import geoio
import json
# Root directory of the project
ROOT_DIR = os.path.abspath("drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/")
DATASET_DIR = os.path.abspath("drive/MyDrive/Mask_RCNN Demo/Mask-RCNN-TF2/building-transfer-learning/building/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

ModuleNotFoundError: ignored

In [ ]:
class SpaceNetConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SpaceNet"
    BACKBONE = "resnet50"
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 building

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_RATIOS = [0.25, 1, 4]

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    USE_MINI_MASK = True

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50

    MAX_GT_INSTANCES=250
    DETECTION_MAX_INSTANCES=350

config = SpaceNetConfig()
config.display()

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.

    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def fill_between(polygon):
    """
    Returns: a bool array
    """
    img = Image.new('1', (650, 650), False)
    ImageDraw.Draw(img).polygon(polygon, outline=True, fill=True)
    mask = np.array(img)
    return mask

class SpaceNetDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    def load_dataset(self, dataset_dir, start=1, end=400):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("SpaceNetDataset", 1, "building")

        # define data locations for images and annotations
        images_dir = os.path.join(dataset_dir, "RGB-PanSharpen1/")
        annotations_dir = os.path.join(dataset_dir, "geojson/buildings/")

        # Iterate through all files in the folder to
        #add class, images and annotaions
        for filename in os.listdir(images_dir)[start:end]:
            image_id  = filename[31:-4]
            image_dir = os.path.join(images_dir,str(filename))
            ann_path  = os.path.join(annotations_dir,"buildings_AOI_2_Vegas_imgg"+str(image_id)+".geojson")
            self.add_image('SpaceNetDataset', image_id=image_id, path=image_dir, annotation=ann_path)


    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        image_dir = os.path.join(DATASET_DIR, "RGB-PanSharpen1/RGB-PanSharpen_AOI_2_Vegas_imgg"+str(image_id)+".png")
        im = Image.open(image_dir)
        return np.asarray(im)

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        masks = np.zeros((650,650))
        ResimPATH = 'D:/DATASET/SpaceNet/Train/AOI_2_Vegas_Train/RGB-PanSharpen/RGB-PanSharpen_AOI_2_Vegas_imgg'+str(image_id)+'.tif'
        RGBTIFResmi = geoio.GeoImage(ResimPATH)

        with open(DATASET_DIR+"/geojson/buildings/buildings_AOI_2_Vegas_imgg"+str(image_id)+".geojson") as f:
            data = json.load(f)
            allBuildings = data['features']

            for building in allBuildings:
                veri = building['geometry']['coordinates'][0]

                tip = str(building['geometry']['type'])
                coordinates = list()
                if tip == ('Point'):
                    continue

                elif tip == ('MultiPolygon'):

                    if isinstance(veri,float): continue

                    kucukBinalar = (building['geometry']['coordinates'])
                    for b in range(len(kucukBinalar)):
                        veri = kucukBinalar[b][0]
                        for i in veri:
                            xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                            xPixel = 649 if xPixel > 649 else xPixel
                            yPixel = 649 if yPixel > 649 else yPixel
                            coordinates.append((xPixel,yPixel))
                else:
                    if isinstance(veri,float): continue

                    for i in veri:
                        xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                        xPixel = 649 if xPixel > 649 else xPixel
                        yPixel = 649 if yPixel > 649 else yPixel
                        coordinates.append((xPixel,yPixel))

                maske = fill_between(coordinates)
                masks = np.dstack((masks,maske))

        if masks.shape != (650,650):
            masks = masks[:,:,1:]
            class_ids = np.asarray([1]*masks.shape[2])
        else:
            class_ids=np.ones((1))
            masks = masks.reshape((650,650,1))
        return masks.astype(np.bool), class_ids.astype(np.int32)

# Training dataset
dataset_train = SpaceNetDataset()
dataset_train.load_dataset(DATASET_DIR,0,3080)
dataset_train.prepare()

# Validation dataset
dataset_val = SpaceNetDataset()
dataset_val.load_dataset(DATASET_DIR,3081,3850)
dataset_val.prepare()

"""
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids,4)

for image_id in image_ids:
    print(image_id)
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
"""
"""
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    #model.load_weights(model.find_last(), by_name=True)
    model.load_weights(model.find_last(), by_name=True)


# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=200,
            layers='all')
"""
#########################################################DETECTION

class InferenceConfig(SpaceNetConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config,
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id,
                            dataset_train.class_names, figsize=(8, 8))

results = model.detect([original_image], verbose=1)
r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'],
                            dataset_val.class_names, r['scores'], ax=get_ax())


"""
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 6)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)

print("mAP: ", np.mean(APs))
"""

## Model Validation

To validate the model, run the following cell

In [ ]:
prediction_file_path = 'drive/My Drive/project_folder/Mask-RCNN-TF2/building-transfer-learning/kangaroo_prediction.py'
%run {prediction_file_path}

# Conclusions

#Future Work

# References

* Bonner, A. (2019). Getting Started With Google Colab. Medium.
https://towardsdatascience.com/getting-started-with-google-colab-f2fff97f594c
* Dair.AI. (2019). Writing Primer for Data Scientists. DAIR Diverse Artificial Intelligence Research Initiative. https://colab.research.google.com/drive/1qi8bXjH389MipsFx3KVQQwGwefDgzzoe
* Elvis. (2018). Primer for Learning Google Colab. Medium. https://medium.com/dair-ai/primer-for-learning-google-colab-bb4cabca5dd6
* Google CoLab. (2018). Google Colaboratory: Welcome: Introduction to Machine Learning Labs. Google. https://colab.research.google.com/drive/1EIB6pAaMM6C2MIdxsrGntYrQoUfcsbmS#forceEdit=true&sandboxMode=true
* Sagar, A. (2019). One-Stop Guide to Google Colab. Medium. https://towardsdatascience.com/one-stop-guide-to-google-colab-d67c94d30516




# Appendices

If you have additional documents that don't fit well directly into the tutorial structure above, you can include links here. This could be more detailed technical documents about your projects, or descriptions of the data set.

# Acknowledgements





<!-- If you're using code, data sets, examples, images or other material based on other people's work, acknowledge it here. This includes parts created with the help of generative AI tools, such as ChatGPT, Google Bard, or Microsoft Copilot. For images and similar material, you should also include a reference anchor or a link to the original.  -->